# Les arbres

Les arbres sont des structures non linéaire qui traduisent une organisation hiérarchique (bases de données, systèmes de fichiers, site web,
analyse syntaxique etc). Les arbres sont largement utilisés en biologie et notamment en génétique avec les arbres phylogénétiques. Ils sont constitués de branches se séparant en n&oelig;ud et se terminant en feuilles. En théorie des graphes les arbres sont des examples de graphes orientés acycliques. C'est à dire qu'ils ont une direction et ne possède pas de circuit.

Les arbres phylogénétiques sont utilisés pour représenter des relations de parenté entres des êtres vivants ou des groupes d'êtres vivants. Chaque n&oelig;ud montrant l'ancêtre commun des différentes branches subsidiaires. De part leurs propriétés, les arbres phylogénétiques ne peuvent prendre en compte les tansferts horizontaux ou hybridations.

Ici nous utiliserons des arbres en format Newick (.nwk), c'est l'un des formats les plus couramment utilisé. Ces arbres contiennent l'information des n&oelig;ud de l'arbre, de leur hiérarchie ainsi que la distance de chacun avec son n&oelig;ud d'ordre supérieur dans un réseau de parenthèses imbriquées.

La hauteur d'un n&oelig;ud est la longueur du plus grand chemin depuis une feuille jusqu'à ce n&oelig;ud. La profondeur d'un n&oelig;ud est la distance avec la racine. L'arité d'un noeud et le nombre de branches successeures qu'il a, l'arité d'un arbre est l'arité maximale de ses noeuds.

<center>
<img src="tree_example.svg">
<center>

# La librairie ETE

[ETE](http://etetoolkit.org) est une librairie python qui facilite la manipulation, analyse et visualisation d'arbres phylogénétiques. Elle permet la manipulation et l'annotation d'arbres particulièrement complexes.

<center>
<img src="https://raw.githubusercontent.com/jhcepas/ete/master/sdoc/gallery.png" width="60%">
<center>

L'avantage d'ETE est qu'il ne fait aucune distinction entre un arbre et un n&oelig;ud. Les arbres pouvant être représentés à partir de leur racine, un n&oelig;ud peut donc devenir la racine d'un arbre. Il est donc très facile de partitionner ou de fusionner des arbres.

Il existe cependant des arbres phylogénétiques non enracinés mais ce n'est pas un problème car ETE considèrera le pointeur du n&oelig;ud ayant la plus grande hauteur comme la racine. Ainsi un arbre non enraciné sera un arbre dont la racine a plus que deux enfants.

In [1]:
%%capture --no-stdout
from ete3 import Tree

In [2]:
unrooted_tree = Tree("(A,B,(C,D));")
print(unrooted_tree)


   /-A
  |
--|--B
  |
  |   /-C
   \-|
      \-D


In [3]:
rooted_tree = Tree("((A,B),(C,D));")
print(rooted_tree)


      /-A
   /-|
  |   \-B
--|
  |   /-C
   \-|
      \-D


Vous pouvez voir que j'ai ici importé la librairie ETE grâce à `import ete3`. Pour l'installer vous pouvez utiliser l'outil de gestion des paquets de votre choix (conda, mamba ou bien pip).

Concernant la génération des arbres phylogénétiques ETE propose une suite d'external apps donnant notamment accès à des commandes bash pour la génération d'arbres. Cependant leur installation est complexe et pose de nombreux problèmes de compatibilité, nous ne vous recommandons donc pas son utilisation. Si vous êtes toutefois interéssés par cela un guide rapide vous est proposé en fin de tuto.

# Nos modules

Nous remplacerons les toolkits d'ETE des modules utilisant principalement la librairie [Biopython](https://biopython.org). Vous les trouverez dans les fichiers joints sous les noms de `sequences_alignment.py` et `tree_generation.py`.

## sequences_alignment

Le premier module sert à faire de l'aligment multiple de séquences tout en gérer plusieurs cas possibles.

In [4]:
import sequences_alignment
help(sequences_alignment)

Help on module sequences_alignment:

NAME
    sequences_alignment - This module align a multi sequences fasta file.

DESCRIPTION
    ClustalW and Muscle algorithms are both supported for the alignment.

FUNCTIONS
    align_seq(in_dir, method='clustalw', out_file=False)
        Align multiple sequences.
        
        Parameters
        ----------
        in_dir : str
            The directory to the file containing the sequences.
        method : str (default clustalw)
            The alignment algorithm, either clustalw or muscle.
        out_dir : boolean (default False)
            Wether or not an output file should be generated.
        
        Returns
        -------
        Bio.Align.MultipleSeqAlignment
            The aligned sequences.

FILE
    d:\data\fac\master\m1\python\ete_tuto\sequences_alignment.py




Cependant son fonctionnement peut se résummer à l'appel d'une command line bash.

### Windows :

Ce module est développé pour fonctionner sous les systèmes Unix, il vous est donc conseillé d'utiliser la commande line sous Windows.
```Bash
./clustalo.exe -i seq_in.fasta -o seq_out.fasta --force
```
Sous windows vous devez spécifier le chemin d'accès vers l'éxécutable du l'algorithme d'alignement, il vous faudra donc le télécharger si vous ne l'avez pas. Vous pouvez télécharger ces éxécutables depuis internet pour [Muscle](https://github.com/rcedgar/muscle/releases/tag/v5.0.1428) et [Clustalw](http://www.clustal.org/download/current/).

### Linux :
```Bash
clustalo -i seq_in.fasta -o seq_out.fasta --force
```

`--force` est utilisé pour forcer l'écrasement du fichier de sortie si il existe déjà.

## tree_generation

Le deuxième module s'occupe de la génération d'arbre phylogénétiques. Ces arbres nécéssitent des séquences alignées, il fait donc appel au module précédent si vous n'avez pas préalablement aligné les séquences d'entrée.

In [5]:
import tree_generation
help(tree_generation)

Help on module tree_generation:

NAME
    tree_generation - This module generate trees based on a multi sequences alignment or does perform the alignment in case it has not been previously done.

DESCRIPTION
    See the documentation of the sequences_alignment module for the alignment.
    
    NJ and UPGMA algorithms are both supported for the tree generation.

FUNCTIONS
    draw_tree(tree)
        Save a phylogenetic tree as a svg file.
        
        Parameters
        ----------
        tree : Bio.Phylo.BaseTree.Tree
            The phylogenetic tree to save.
    
    tree_constructor(seq, spike708=False, svg=False)
        Construct a tree using the NJ or UPGMA method.
        
        Parameters
        ----------
        seq : Bio.Align.MultipleSeqAlignment
            A multiple sequences alignement.
        spike708 : boolean (default False)
            If the data provided are the original spike data.
        svg : boolean (default False)
            Wether or not the tree 

# Le jeu de donnée

Pour mettre en pratique les outils d'ETE nous étudierons un jeu séquences de la protéine Spike du SARS-COV2. Ce jeu de donnée à été récupéré depuis la base de donnée du NCBI. Afin de trier ces séquences un threshold du nombre d'apparition de ces séquences a appliqué. Cela nous donne le fichier fasta que vous trouverez en `data\spike_data_708.fasta`

Le fichier fasta à ensuite été aligné à l'aide du module `sequences_alignment` présenté ci-dessus et en utilisant Clustal$\Omega$. Un arbre a ensuite été généré à partir de cet alignement et grâce au module `tree_generation`. Nous avons utilisé blosum90 comme matrice de substitution à cause de la proximité des séquences. Nous avons utilisé le Neighbor Joining comme méthode phénétique.

# Ressources

Voici quelques ressources intéréssantes qui vous permettront d'approfondir le sujet.

Les ressources du projet ETE :
* Tutoriel - http://etetoolkit.org/docs/latest/tutorial/index.html
* Cookbook - http://etetoolkit.org/cookbook/
* Repository GitHub - https://github.com/etetoolkit/ete
* Article de recherche - https://academic.oup.com/mbe/article/33/6/1635/2579822

Autres ressources sur ETE :
* Tutoriel - https://evomics.org/wp-content/uploads/2019/01/ete_tutorial.pdf
* Cours pratique - https://github.com/etetoolkit/course

Les ressources de Biopython :
* Tree Construction submodule - https://biopython.org/docs/1.75/api/Bio.Phylo.TreeConstruction.html
* Tutoriel et CookBook - https://biopython.org/DIST/docs/tutorial/Tutorial.html

Tous les fichiers utilisés sont disponibles [ici](https://github.com/nobabar/ete3_howto).

# Pour aller plus loin : les external apps

<div class="alert alert-block alert-warning">
<b>Attention :</b> la lecture de ce chapître n'est pas nécéssaire à la compréhension de notre projet ni même de l'utilisation de la librairie ETE. De plus elle suppose des pré-requis et ne rentrera pas dans tous les détails. Elle est donc uniquement faites pour prévenir les différents problèmes pouvant survenir lors de l'installation et l'utilisation des externals apps d'ETE.

De plus nous ne vous conseillons pas de vous lancer dans l'installation des ces external apps si vous ne possédez pas une distribution "classique" de linux. En effet de nombreux problèmes sont survenus lorsque nous avons essayé d'installer celles-ci sur WSL ou Fedora.
</div>

## Installation

Nous prendrons ici l'example d'une installation avec `conda` mais il est aussi possible d'utiliser `mamba` ou `docker`.

Création et activation d'un envirronement virtuel conda :
```Bash
$ conda create -name ete3_env python=X.X
```

```Bash
$ conda activate ete3_env
```

Vous pouvez optionnelement préciser la version de python. Les external apps sont optimisée pour python 2.7 mais il est possible de les utiliser jusqu'à python 3.4. Il vous fera donc sûrement essayer plusieurs version avant que tout marche correctement.

Vous pouvez ensuite installer l'ensemble des external apps ou bien seulement la toolchain (requise pour la génération d'arbre).
```Bash
$ conda install -c etetoolkit ete3 ete3_external_apps
```

```Bash
$ conda install -c etetoolkit ete3 ete_toolchain
```

Il est possible que l'installation ne fonctionne pas à cause de conflits, il vous faudra alors vérifier que vous avez la bonne version de `PyQt`.
```Bash
$ conda list
```

Si vous avez la version 5 de `PyQt` il vous faudra installer la 4.
```Bash
$ conda install qt=4
```

Vérification de l'installation :
```Bash
$ ete3 build check
```

## Construction des arbres
 
```Bash
$ ete3 build -w workflow -a input_file -o output_folder
```

Une liste des workflows est disponible [ici](http://etetoolkit.org/cookbook/ete_build_list_apps.ipynb).

Il est possible que ETE vous demande un serveur pour éxécuter cette commande. Pour cela vous pouvez faire appel à un [display server](https://www.wikiwand.com/en/Windowing_system#/Display_server) ou à un serveur distant.

### Sur display server

La solution la plus courant est d'utiliser [Xvfb](https://www.x.org/releases/X11R7.7/doc/man/man1/Xvfb.1.xhtml).
```Bash
$ xvfb-run ete3 build -w standard_raxml -a spike_data_708.fasta -o output_tree
```

Mais il est tout à fait possible d'utiliser un serveur [X.Org](https://gitlab.freedesktop.org/xorg/xserver) tel que sa version avec dummy devices, [Xpra](https://github.com/Xpra-org/xpra).


### Sur serveur distant

Nous prendrons ici l'exemple d'une utilisation sur les [clusters de l'IFB](https://www.france-bioinformatique.fr/cluster-ifb-core/) et donc plus géénralement du système de soumission SLURM.

#### Commandes générales
```Bash
$ ssh <username>@core.cluster.france-bioinformatique.fr    #connect
```

```Bash
$ module av                                                #check available modules
```

```Bash
$ cd <dir>                                                 #move to a directory
```

```Bash
$ ls                                                       #check current directory
```

```Bash
$ nano <file>                                              #edit a file using nano text editor
```

#### Transfert de données

Transfert de données locales au serveur par SSH :
```Bash
$ scp -r <data_dir> <username>@core.cluster.france-bioinformatique.fr:.
```

Dans l'autre sens il existe deux solutions :
* Clients FTP/SFTP FileZilla/Cyberduck/...
* Connecter l'explorateur de fichier à votre adresse sur le serveur `ftp://<username>@core.cluster.france-bioinformatique.fr`

#### Soummettre une tâche

```Bash
$ nano tree.sh    # Création d'un fichier de commande bash contenant vos instructions

__________________________________________________________________________________
#!/bin/bash
################################## Slurm options #################################
### Job name
#SBATCH --job-name=tree_generation
### Limit run time "days-hours:minutes:seconds"
#SBATCH --time=01:00:00
### Requirements
#SBATCH --partition=fast
#SBATCH --nodes=1
#SBATCH --ntasks-per-node=1
#SBATCH --mem-per-cpu=8GB
##################################################################################
module load python/2.7
ete3 build -w standard_raxml -a spike_data_708.fasta -o output_tree
__________________________________________________________________________________
```

```Bash
$ sbatch tree.sh           # lancement la tâche
```

```Bash
$ nano slurm-<jobID>.out    # vérification du log
```